Protein Structure Prediction. Task: Predict the 3D structure of a given protein sequence
using homology modeling or threading techniques. Deliverable: A report presenting the
predicted protein structure, along with an analysis of its potential functions and interactions

In [1]:
import numpy as np
from Bio import SeqIO, Align
from Bio.PDB import *
import warnings
from Bio import BiopythonWarning

# Suppress Biopython warnings
warnings.simplefilter('ignore', BiopythonWarning)

In [3]:
aa_properties = {
    'A': {'hydrophobicity': 1.8, 'volume': 88.6, 'polarity': 0},
    'R': {'hydrophobicity': -4.5, 'volume': 173.4, 'polarity': 1},
    'N': {'hydrophobicity': -3.5, 'volume': 114.1, 'polarity': 1},
    # ... (other amino acids would be defined similarly)
}

# Secondary structure propensities
ss_propensities = {
    'A': {'helix': 1.42, 'sheet': 0.83, 'coil': 0.9},
    'R': {'helix': 1.21, 'sheet': 0.93, 'coil': 0.99},
    'N': {'helix': 0.67, 'sheet': 0.89, 'coil': 1.33},
    # ... (other amino acids would be defined similarly)
}

In [4]:
def search_template_database(query_sequence):
    """Simulate searching for template structures in the PDB database."""
    mock_template = {
        'pdb_id': '1ABC',
        'sequence': 'SIMILAR_SEQUENCE',
        'resolution': 2.5,
        'coordinates': np.random.rand(len(query_sequence), 3)
    }
    return mock_template

In [5]:
def align_sequences(query_sequence, template_sequence):
    """Perform sequence alignment between query and template."""
    aligner = Align.PairwiseAligner()
    aligner.open_gap_score = -10
    aligner.extend_gap_score = -0.5
    alignments = aligner.align(query_sequence, template_sequence)
    return alignments[0]

In [6]:
def predict_secondary_structure(sequence):
    """Predict secondary structure using propensities."""
    structure = []
    window_size = 5
    padded_seq = 'X' * (window_size // 2) + sequence + 'X' * (window_size // 2)
    
    for i in range(len(sequence)):
        window = padded_seq[i:i + window_size]
        helix_score = sheet_score = coil_score = 0

        # Calculate propensity scores
        for aa in window:
            if aa in ss_propensities:
                helix_score += ss_propensities[aa]['helix']
                sheet_score += ss_propensities[aa]['sheet']
                coil_score += ss_propensities[aa]['coil']

        # Assign structure based on highest score
        if max(helix_score, sheet_score, coil_score) == helix_score:
            structure.append('H')
        elif max(helix_score, sheet_score, coil_score) == sheet_score:
            structure.append('E')
        else:
            structure.append('C')
    
    return ''.join(structure)

In [7]:
def build_backbone(sequence, secondary_structure):
    """Build basic backbone structure using predicted secondary structure."""
    coordinates = np.zeros((len(sequence), 3))
    current_pos = np.array([0.0, 0.0, 0.0])
    
    for i in range(len(sequence)):
        if secondary_structure[i] == 'H':  # Helix
            current_pos += np.array([1.5, 0.5, 0.5])
        elif secondary_structure[i] == 'E':  # Sheet
            current_pos += np.array([1.5, 0.0, 0.0])
        else:  # Coil
            current_pos += np.array([1.0, np.random.rand(), np.random.rand()])
        coordinates[i] = current_pos
    
    return coordinates

In [8]:
def refine_structure(coordinates):
    """Simple structure refinement."""
    refined_coords = coordinates + np.random.normal(0, 0.1, coordinates.shape)
    return refined_coords

In [9]:
def save_structure(coordinates, sequence, filename):
    """Save predicted structure in PDB format."""
    structure = Structure.Structure('predicted')
    model = Model.Model(0)
    chain = Chain.Chain('A')
    
    for i, (coord, aa) in enumerate(zip(coordinates, sequence)):
        residue = Residue.Residue((' ', i, ' '), aa, '')
        atom = Atom.Atom('CA', coord, 20.0, 1.0, ' ', 'CA', i, 'C')
        residue.add(atom)
        chain.add(residue)
    
    model.add(chain)
    structure.add(model)
    
    io = PDBIO()
    io.set_structure(structure)
    io.save(filename)

In [10]:
def predict_structure(sequence):
    """Main function to predict protein structure."""
    # Step 1: Search for template
    template = search_template_database(sequence)
    
    # Step 2: Align sequences
    alignment = align_sequences(sequence, template['sequence'])
    
    # Step 3: Predict secondary structure
    secondary_structure = predict_secondary_structure(sequence)
    
    # Step 4: Build initial backbone
    initial_coords = build_backbone(sequence, secondary_structure)
    
    # Step 5: Refine structure
    final_coords = refine_structure(initial_coords)
    
    return {
        'coordinates': final_coords,
        'secondary_structure': secondary_structure,
        'alignment': alignment,
        'template_used': template['pdb_id']
    }

In [11]:
def main():
    """Example usage."""
    sequence = "MKWVTFISLLLLFSSAYSRGVFRRDAHKSEVAHRFKDLGEENFKALVLIAFAQYLQQCP"
    
    result = predict_structure(sequence)
    
    save_structure(
        result['coordinates'],
        sequence,
        'predicted_structure.pdb'
    )
    
    print(f"Secondary structure prediction: {result['secondary_structure']}")
    print(f"Template used: {result['template_used']}")
    print("Structure has been saved to 'predicted_structure.pdb'")


In [12]:
if __name__ == "__main__":
    main()

Secondary structure prediction: HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCHHHHHHHHHHHHHHH
Template used: 1ABC
Structure has been saved to 'predicted_structure.pdb'


Viva Questions and Answers
1. What is protein structure prediction?
Answer: Protein structure prediction involves determining the 3D structure of a protein based on its amino acid sequence. It is essential for understanding the function of the protein and for applications like drug design.
2. What are the four levels of protein structure?
Answer:
Primary Structure: Linear sequence of amino acids.
Secondary Structure: Local folding into structures like alpha-helices and beta-sheets.
Tertiary Structure: 3D folding of the entire polypeptide chain.
Quaternary Structure: Assembly of multiple polypeptide chains.
3. Why is sequence alignment important in protein modeling?
Answer: Sequence alignment helps identify similar sequences in known structures (templates), which can be used to model the unknown structure of a target protein.
4. What is secondary structure prediction?
Answer: It predicts regions of a protein sequence that form alpha-helices, beta-sheets, or random coils based on the propensity of each amino acid to form these structures.
5. What is the significance of using a sliding window in secondary structure prediction?
Answer: A sliding window allows evaluating a segment of the sequence to predict the structure based on surrounding amino acids, providing context for accurate prediction.
6. How is the 3D structure of a protein typically represented?
Answer: The 3D structure is represented by the spatial coordinates of atoms, often visualized using PDB files that list atom positions.
7. What is a PDB file?
Answer: A Protein Data Bank (PDB) file is a standard format for representing 3D structures of proteins, nucleic acids, and complex assemblies. It contains atomic coordinates, connectivity, and other structural information.
8. What is the role of AgglomerativeClustering in this context?
Answer: This is not used in the protein structure prediction code. However, in data clustering contexts, Agglomerative Clustering is a hierarchical clustering method.
9. Why do we refine the structure after building the initial backbone?
Answer: Refinement helps minimize steric clashes and brings the model closer to a physically realistic conformation by simulating energy minimization.
10. How would you improve the accuracy of this prediction model?
Answer: Improvements could include using more sophisticated secondary structure prediction methods, incorporating energy-based refinement algorithms, and using databases with a larger set of high-resolution templates.